In [ ]:
# | default_exp _code_generator.app_skeleton_generator

In [ ]:
# | export

from typing import *
import time
import json
from pathlib import Path
from collections import defaultdict

from yaspin import yaspin

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.chat import CustomAIChat, ValidateAndFixResponse
from faststream_gen._code_generator.helper import (
    write_file_contents,
    validate_python_code,
    retry_on_error,
)
from faststream_gen._code_generator.prompts import APP_SKELETON_GENERATION_PROMPT
from faststream_gen._code_generator.constants import (
    STEP_LOG_DIR_NAMES,
    APPLICATION_FILE_PATH,
    LOGS_DIR_NAME
)

In [ ]:


from tempfile import TemporaryDirectory

import pytest

from faststream_gen._components.logger import suppress_timestamps
from faststream_gen._code_generator.constants import OpenAIModel
from faststream_gen._code_generator.helper import mock_openai_create

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


def _validate_response(
    response: str, output_directory: str, **kwargs: Dict[str, Any]
) -> List[str]:
    target_file_name = Path(output_directory) / APPLICATION_FILE_PATH
    write_file_contents(str(target_file_name), response)
    ret_val = validate_python_code(str(target_file_name))

    return ret_val

In [ ]:
fixture = """
import os
def say_hello():
    print("hello")
"""

with TemporaryDirectory() as d:
    actual = _validate_response(fixture, d)
    expected = []

    print(actual)
    assert actual == expected

[]


In [ ]:
fixture = """
import os
import invalid_module
def say_hello():
    print("hello")
"""

with TemporaryDirectory() as d:
    actual = _validate_response(fixture, d)
    expected = ["ModuleNotFoundError: No module named 'invalid_module'"]

    print(actual)
    assert actual == expected

["ModuleNotFoundError: No module named 'invalid_module'"]


In [ ]:
# | export


@retry_on_error()  # type: ignore
def _generate(
    model: str,
    prompt: str,
    app_description_content: str,
    total_usage: List[Dict[str, int]],
    output_directory: str,
    **kwargs,
) -> Tuple[str, List[Dict[str, int]]]:
    app_generator = CustomAIChat(
        params={
            "temperature": 0.2,
        },
        model=model,
        user_prompt=prompt,
        #         semantic_search_query=app_description_content,
    )
    app_validator = ValidateAndFixResponse(app_generator, _validate_response)
    validator_result = app_validator.fix(
        app_description_content,
        total_usage,
        STEP_LOG_DIR_NAMES["skeleton"],
        str(output_directory),
        **kwargs,
    )

    return (
        (validator_result, True)
        if isinstance(validator_result[-1], defaultdict)
        else validator_result
    )

In [ ]:
model = OpenAIModel.gpt3.value
prompt = "Some valid prompt"
app_description_content = "some valid app description"
total_usage = [defaultdict(int)]


test_response = """
print("some valid skeleton code")
"""

with TemporaryDirectory() as d:
    with mock_openai_create(test_response):
        total_usage, is_valid_skeleton_code = _generate(
            model, prompt, app_description_content, total_usage, d
        )
        assert is_valid_skeleton_code == True
        print("OK")

[INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

You will encounter sections marked as:

==== APP DESCRIPTION: ====

These sections contain the description of the FastStream app you need to implement. Treat everything below this line, until the end of the prompt, as the description to follow for the app implementation.


In [ ]:
model = OpenAIModel.gpt3.value
prompt = "Some invalid prompt"
app_description_content = "some invalid app description"
total_usage = [defaultdict(int)]

test_response = """
print("some valid skeleton code"
"""

with TemporaryDirectory() as d:
    with mock_openai_create(test_response):
        total_usage, is_valid_skeleton_code = _generate(
            model, prompt, app_description_content, total_usage, d
        )
        print(is_valid_skeleton_code)
        assert is_valid_skeleton_code == False
        print("OK")

[INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

You will encounter sections marked as:

==== APP DESCRIPTION: ====

These sections contain the description of the FastStream app you need to implement. Treat everything below this line, until the end of the prompt, as the description to follow for the app implementation.


[INFO] faststream_gen._code_generator.chat: Validation failed, trying again...Errors:
SyntaxError: '(' was never closed (application.py, line 2)
[INFO] faststream_gen._code_generator.helper: Attempt 1 failed. Restarting step.
[INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

You will encounter sections marked as:

==== A

In [ ]:
# | export


def generate_app_skeleton(
    validated_description: str,
    output_directory: str,
    model: str,
    total_usage: List[Dict[str, int]],
    relevant_prompt_examples: str,
) -> List[Dict[str, int]]:
    """Generate skeleton code for the new FastStream app from the application description

    Args:
        code_gen_directory: The directory containing the generated files.
        total_usage: list of token usage.
        relevant_prompt_examples: Relevant examples to add in the prompts.

    Returns:
        The total token used to generate the FastStream code
    """
    logger.info("==== Description to Skeleton Generation ====")
    with yaspin(
        text=f"Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)...",
        color="cyan",
        spinner="clock",
    ) as sp:
        prompt = APP_SKELETON_GENERATION_PROMPT.replace(
            "==== RELEVANT EXAMPLES GOES HERE ====", f"\n{relevant_prompt_examples}"
        )

        total_usage, is_valid_skeleton_code = _generate(
            model, prompt, validated_description, total_usage, output_directory
        )
        sp.text = ""
        if is_valid_skeleton_code:
            message = f" ✔ FastStream app skeleton code generated."
        else:
            message = " ✘ Error: Failed to generate a valid application skeleton code."
            sp.color = "red"

        sp.ok(message)
        return total_usage, is_valid_skeleton_code

In [ ]:
fixture_description = "Some valid description"

relevant_prompt_examples = "Some valid examples"

fixture_response = 'print("hi")'

with mock_openai_create(fixture_response):
    with TemporaryDirectory() as d:
        output_file = Path(d) / APPLICATION_FILE_PATH

        usage, is_valid_skeleton_code = generate_app_skeleton(
            fixture_description, d, OpenAIModel.gpt3.value, [], relevant_prompt_examples
        )

        assert Path(output_file).exists()

        with open(output_file, "r", encoding="utf-8") as f:
            actual = f.read()

        print(actual)
        assert actual == fixture_response
        
        assert is_valid_skeleton_code

[INFO] __main__: ==== Description to Skeleton Generation ====
⠋ Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)...[INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

You will encounter sections marked as:

==== APP DESCRIPTION: ====

These sections contain the description of the FastStream a

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:119: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


In [ ]:
fixture_description = "Some invalid description"

relevant_prompt_examples = "Some invalid examples"

fixture_response = 'i am a not a python code'

with mock_openai_create(fixture_response):
    with TemporaryDirectory() as d:
        output_file = Path(d) / APPLICATION_FILE_PATH
        usage, is_valid_skeleton_code = generate_app_skeleton(
            fixture_description, d, OpenAIModel.gpt3.value, [], relevant_prompt_examples
        )
        print(is_valid_skeleton_code)
        assert not is_valid_skeleton_code

[INFO] __main__: ==== Description to Skeleton Generation ====
⠋ Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)...[INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

You will encounter sections marked as:

==== APP DESCRIPTION: ====

These sections contain the description of the FastStream a

⠹ Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)... [INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

You will encounter sections marked as:

==== APP DESCRIPTION: ====

These sections contain the description of the FastStream app you need to implement. Treat everything below this line, u

⠼ Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)... [INFO] faststream_gen._code_generator.chat: 

Prompt to the model: 

===Role:system===

Message:

You are an expert Python developer, tasked to generate executable Python code as a part of your work with the FastStream framework. 

You are to abide by the following guidelines:

1. You must never enclose the generated Python code with ``` python. It is mandatory that the output is a valid and executable Python code. Please ensure this rule is never broken.

2. Some prompts might require you to generate code that contains async functions. For example:

async def app_setup(context: ContextRepo):
    raise NotImplementedError()

In such cases, it is necessary to add the "import asyncio" statement at the top of the code. 

You will encounter sections marked as:

==== APP DESCRIPTION: ====

These sections contain the description of the FastStream app you need to implement. Treat everything below this line, u

⠧ Generating FastStream app skeleton code (usually takes around 15 to 45 seconds)... ************************************************************************************************************************
is_valid_skeleton_code=False
total_usage=[defaultdict(<class 'int'>, {'prompt_tokens': 387, 'completion_tokens': 3, 'total_tokens': 390}), defaultdict(<class 'int'>, {'prompt_tokens': 387, 'completion_tokens': 3, 'total_tokens': 390}), defaultdict(<class 'int'>, {'prompt_tokens': 387, 'completion_tokens': 3, 'total_tokens': 390})]
 ✘ Error: Failed to generate a valid application skeleton code.                      
False


/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:228: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(value) if value else value
